In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KNOW기반 직업 추천 알고리즘 경진대회 Private Board 4위(Macro F1 score : 0.74523) thingsu 팀 제출용으로 정리한 코드입니다.   
(코드는 colab 기준으로 작성하였으며, 노트 설정을 가속기 : GPU, RAM : 표준으로 설정해주세요) 

솔루션의 개요입니다. 

- Feature : 
    - 설문지의 문항을 numeric feature와 categorical feature로 나누어 적용
    - 자연어 답변에 대해서 bag of words로 feature를 생성
- Model :
    - RandomForest : 5 fold Ensemble(soft voting)과 Full data 2 가지 모델을 얻기
    - Neural Network : Full data에 대해서 parameter와 augment 여부를 달리 해 3 가지 모델을 얻기
- Ensemble :
    - 위에서 얻은 5개의 inference 결과에 대해서 Hard Voting 방식으로 Ensemble

실행 방법은 :
1. 위의 함수들을 모두 define 합니다.
2. 각 결과가 저장될 output directory를 적절하게 설정하고 실행 코드 부분을 실행해 5개 result를 얻습니다.
3. Ensemble 부분을 실행해 최종 결과를 얻습니다.

# Install

## DL

In [ ]:
!pip install gensim 

## Mecab

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Processing /tmp/mecab-python-0.996
  Created wheel for mecab-python: filename=mecab_python-0.996_ko_0.9.2-cp37-cp37m-linux_x86_64.whl size=141818 sha256=d07bf41e361d1d64deaedc69ab6b45525d1ea5c50fe630adfb6e69f5373b43e2
  Stored in directory: /root/.cache/pip/wheels/40/7b/9f/2922869bef86c3354ae7034f7a3647c573ee1997c2dad0290a
Failed to build mecab-python
  Attempting uninstall: mecab-python
    Found existing installation: mecab-python 0.996-ko-0.9.2
    Uninstalling mecab-python-0.996-ko-0.9.2:
      Successfully uninstalled mecab-python-0.996-ko-0.9.2
    Running setup.py install for mecab-python: started
    Running setup.py install for mecab-python: finished with status 'done'


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  DEPRECATION: mecab-python was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


# Import

In [ ]:
import random
import pandas as pd
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from collections import defaultdict, Counter

import os
import gc
import time
import pickle
import warnings

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain

from konlpy.tag import Okt
from gensim.models import Word2Vec

import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader

warnings.filterwarnings(action='ignore')

In [ ]:
# 상수
data_dir = "/content/drive/MyDrive/DS/know/job/KNOW_data/"
years = ['2017', '2018', '2019', '2020']

nl_ans = {
    0 : ['bq4_1a','bq4_1b','bq4_1c','bq5_2','bq19_1','bq30','bq31','bq32','bq33','bq34','bq38_1'],
    1 : ['bq4_1a','bq4_1b','bq4_1c','bq5_2','bq28_1','bq29','bq30','bq31','bq32','bq33','bq37_1'],
    2 : ['bq4_1a','bq4_1b','bq4_1c','bq5_2','bq18_10','bq20_1','bq22','bq23','bq24','bq27_1'],
    3 : ['bq4_1a','bq4_1b','bq4_1c','bq5_2','bq18_10','bq20_1','bq26_1']
}

idx2col ={
    '성별' : {'2017':'bq36','2018':'bq35','2019':'bq25','2020':'bq24'},
    '산업유형' : {'2017':'bq1','2018':'bq1','2019':'bq1','2020':'bq1'},
    '자격증' : {'2017':'bq4_1a','2018':'bq4_1a','2019':'bq4_1a','2020':'bq4_1a'},
    '전공' : {'2017':'bq38_1','2018':'bq37_1','2019':'bq27_1','2020':'bq26_1'},
    '훈련' : {'2017':'bq5_2','2018':'bq5_2','2019':'bq5_2','2020':'bq5_2'},
    '유사직업' : {'2017':'bq30','2018':'bq29','2019':None,'2020':None},
    '일자리변화' : {'2017':'bq19_1','2018':'bq28_1','2019':'bq20_1','2020':'bq20_1'},
    '사용도구' : {'2017':'bq31','2018':'bq30','2019':None,'2020':None},
    '이전직업' : {'2017':'bq32','2018':'bq31','2019':'bq22','2020':None},
    '전직가능' : {'2017':'bq33','2018':'bq32','2019':'bq23','2020':None},
    '신직업' : {'2017':'bq34','2018':'bq33','2019':'bq24','2020':None},
    '변화예시' : {'2017':None,'2018':None,'2019':'bq18_10','2020':'bq18_10'},
}

# Preprocessing

## Data Setting

In [ ]:
def erase_na(know_data):
    # 밀려서 학습에 방해될 거라고 판단된 데이터 삭제
    for i in range(4):
        print(f"erase na row {know_data[i][know_data[i].isna().any(axis=1)].shape[0]}")
        know_data[i] = know_data[i].dropna(axis=0)

    for d, r in zip([1,1,2],[3064,8551,1302]): 
        try :
            know_data[d] = know_data[d].drop([r])
        except : 
            print(f"already drop {r} in {d}")
    for i in range(4):
        print(sum(know_data[i].isnull().sum()))

    return know_data

def null_check(know_data, know_test):
    # Null 값 존재 여부 체크 및 값 변경
    nm_ans = {}
    for i in range(4):
        nm_ans[i] = [c for c in list(know_data[i].columns) if c not in nl_ans[i]]
        # numeric 변수
        for c in nm_ans[i]:
            know_data[i][c] = know_data[i][c].replace([' ', ''], '0')
            if c != 'knowcode' and c != 'kfold':
                know_test[i][c] = know_test[i][c].replace([' ', ''], '0')
        # text 변수
        know_data[i][nl_ans[i]] = know_data[i][nl_ans[i]].replace([np.nan, ' ', '없다','없음','모름','무'], '')
        know_test[i][nl_ans[i]] = know_test[i][nl_ans[i]].replace([np.nan, ' ', '없다','없음','모름','무'], '')

        print(f'{i} train data null value check', sum(know_data[i].isnull().sum()))
        print(f'{i} test data null value check', sum(know_test[i].isnull().sum()))

    return know_data, know_test

def data_setting(know_data, know_test):
    # na 값 
    know_data = erase_na(know_data)
    # 값 대체 하기
    know_data, know_test = null_check(know_data, know_test)
    # 오타 수정
    know_data[0]['aq41_1'] = know_data[0]['aq41_1'].replace(33,3)
    
    for c in know_data[1].columns : 
        if len(c)==5 and c[3]!='_' :
            know_data[1][c] = know_data[1][c].astype(float).astype(int).apply(lambda x : 1 if x > 0 else 0)
            know_test[1][c] = know_test[1][c].astype(float).astype(int).apply(lambda x : 1 if x > 0 else 0)

    print("done!")
    return know_data, know_test

def df2dict(dfs, vartype_info_):
    for i in range(4) :
        temp = {}
        for c in ['C','NM','NL']:
            temp[c] = dfs[i][vartype_info_[i][c]]
        if 'knowcode' in dfs[i].columns :
            temp['y'] = dfs[i].knowcode
        dfs[i] = temp
    return dfs

def train_encode(datas):
    year_encoder = {}
    for year, df in zip(years, datas):

        encoders = {}

        for col in df['NM'].columns:
            try:
                df['NM'][col] = df['NM'][col].astype(float).astype(int)
            except:
                print(f'{year} {col} NM error!!')

        for col in df['C'].columns:
            encoder = LabelEncoder()
            df['C'][col] = df['C'][col].map(str)
            df['C'][col] = encoder.fit_transform(df['C'][col])
            encoders[col] = encoder

        year_encoder[year] = encoders

    return datas, year_encoder

def test_encode(datas, encoders):
  for year, df in zip(years, datas):

    for col in df['NM'].columns:
        try:
            df['NM'][col] = df['NM'][col].astype(float).astype(int)
        except:
            df['NM'][col] = df['NM'][col].astype(str).apply(lambda x : int(x) if x.isnumeric() else 0)

    for col in df['C'].columns:
        try : 
            encoder = encoders[year][col]
            df['C'][col] = df['C'][col].map(str)
            category_map = {category: idx for idx, category in enumerate(encoder.classes_)}
            df['C'][col] = df['C'][col].apply(lambda x: category_map[x] if x in category_map else -1)
        except :
            print(f'{year} {col} C error!!')

  return datas

def get_cate_info(year_encoder):
    cate_info = {}
    for year in years :
        cate_info[year] = {'col_name' : [], 'n_class' : []}
        for c in year_encoder[year]:
            le = year_encoder[year][c]
            cate_info[year]['col_name'].append(c)
            cate_info[year]['n_class'].append(len(le.classes_)+1) # -1 까지
    return cate_info

def Q(x, Q1, Q2, Q3):
  if x == 0 : return 0
  if x < Q1 : return 1
  if x < Q2 : return 2
  if x < Q3 : return 3
  return 4

def salary_preprocessing(know_data, know_test, vartype_info):
   # 임시 연봉 처리용
    for i in range(4):
        col = vartype_info[i]['NM'][-3:]
        for c in col :
            sts = know_data[i]['NM'][c][know_data[i]['NM'][c]>0].describe()
            Q1, Q2, Q3 = sts['25%'], sts['50%'], sts['75%']
            # 추가
            know_data[i]['C'][f"{c}_c"] = know_data[i]['NM'][c].apply(lambda x : Q(x, Q1, Q2, Q3))
            know_test[i]['C'][f"{c}_c"] = know_test[i]['NM'][c].apply(lambda x : Q(x, Q1, Q2, Q3))
            # 삭제
            know_data[i]['NM'] = know_data[i]['NM'].drop(c,axis=1)
            know_test[i]['NM'] = know_test[i]['NM'].drop(c,axis=1)
            # 정보 변경
            vartype_info[i]['NM'].remove(c)
            vartype_info[i]['C'].append(f"{c}_c")


    return know_data, know_test, vartype_info

## Bag of Words

In [ ]:
from konlpy.tag import Mecab

tokenizer = Mecab()

def get_tokenized_data(know_data, know_test) :
    path = os.path.join(data_dir,'morphs_.pickle')
    if not os.path.exists(path) :
        dta, tst = defaultdict(dict), defaultdict(dict)
        for i in range(4):
            for col in know_data[i]['NL'].columns :
                dta[i][col] = know_data[i]['NL'][col].apply(lambda x : tokenizer.morphs(str(x)))
                tst[i][col] = know_test[i]['NL'][col].apply(lambda x : tokenizer.morphs(str(x)))
            
        with open(os.path.join(data_dir,'morphs_.pickle'),'wb') as f :
            pickle.dump([dta,tst],f)

In [ ]:
def tokenizing(text):
  words = text.split('#')
  return words

def get_nl_features(feature_per_column = 600, tokenize = 'morphs',  min_df=None):
  # Column 별로 bag of word 방식으로로 feature를 생성
  # 최소 등장 빈도(min_df), Column 별 bag of word의 size (feature_per_column), end 
  nl_encoders, tr_X, ts_X = defaultdict(dict), defaultdict(dict), defaultdict(dict)
  # 사전에 전처리한 파일 로딩
  with open(os.path.join(data_dir,f'asset/nl_{tokenize}.pickle'),'rb') as f :
    dta, tst = pickle.load(f)

  for j, nl_col in nl_ans.items():
    for col_name in nl_col :

      if col_name == 'bq4_1b' or col_name == 'bq4_1c':
        continue

      if col_name == 'bq4_1a':
        tr_col = dta[j]['bq4_1a']+dta[j]['bq4_1b']+dta[j]['bq4_1c']
        ts_col = tst[j]['bq4_1a']+tst[j]['bq4_1b']+tst[j]['bq4_1c']

      else :
        tr_col = dta[j][col_name]
        ts_col = tst[j][col_name]

      if min_df is None :
      # min_df 미설정시 column별로 feature_per_column에 맞는 값으로 설정
        end = 12
        for i in range(1,end):
          ct = CountVectorizer(tokenizer = lambda x: tokenizing(x), min_df = i,stop_words=['','(',')',',','.','/'])

          rrr = ct.fit_transform(tr_col.apply(lambda x : '#'.join(list(x))))
          
          # vocab size가 feature_per_column 보다 작거나, min_df의 상한일 때  
          if len(ct.vocabulary_) > feature_per_column :
            if i != (end-1):
              continue
              
          ttt = ct.transform(ts_col.apply(lambda x : '#'.join(list(x))))

          tr_X[j][col_name], ts_X[j][col_name] = rrr, ttt
          print(j,col_name,'min_df =', i, 'vocab size', len(ct.vocabulary_))
          nl_encoders[j][col_name] = ct
          break

      else :

        ct = CountVectorizer(tokenizer = lambda x: tokenizing(x), min_df = min_df, stop_words=['','(',')',',','.','/'])

        rrr = ct.fit_transform(tr_col.apply(lambda x : '#'.join(list(x))))            
        ttt = ct.transform(ts_col.apply(lambda x : '#'.join(list(x))))

        tr_X[j][col_name], ts_X[j][col_name] = rrr, ttt
        print(j,col_name,'min_df =', min_df, 'vocab size', len(ct.vocabulary_))
        nl_encoders[j][col_name] = ct

  return nl_encoders, tr_X, ts_X 

def get_bow(nls, using_nl, nl_encoders):
  # column_word 꼴로 formatting해서 저장 ex) 자격증_toeic, ...
  datas = {}
  for i, year in enumerate(years): 

    temp, result = [], {}
    for col in using_nl:
      if idx2col[col][year] is not None :
        x = nls[i][idx2col[col][year]]
        feature_names = [f'{col}_{c}' for c in nl_encoders[i][idx2col[col][year]].get_feature_names()]
        temp.append(pd.DataFrame(x.todense(),columns=feature_names))

    X = pd.concat(temp,axis=1)

    datas[year] = X
    
  return datas

## total step code

In [ ]:
def get_data(training_args):
    know_data_ = [pd.read_csv(path, index_col=False) for path in sorted(glob(os.path.join(data_dir,'train/data/*.csv')))]
    know_test_ = [pd.read_csv(path, index_col=False) for path in sorted(glob(os.path.join(data_dir,'test/data/*.csv')))]

    know_data_, know_test_ = data_setting(know_data_, know_test_)

    # 변수 종류 정리 
    vartype_info_ = [pd.read_csv(path, header = None, index_col=False) for path in sorted(glob(os.path.join(data_dir,'asset/List_csv*.csv')))]
    vartype_info_ = [vartype_info_[i].groupby(1).agg(lambda x : list(x)).to_dict()[0] for i in range(4)]
    for i in range(4):
        nm_col = [c for c in know_data_[i].columns if (c[0] not in ['b','k', 'i'])]
        vartype_info_[i]['NM'] = nm_col + vartype_info_[i]['NM']

    know_data_, know_test_ = df2dict(know_data_, vartype_info_), df2dict(know_test_,vartype_info_)

    know_data_, year_encoder = train_encode(know_data_)
    know_test_ = test_encode(know_test_, year_encoder)
    cate_info = get_cate_info(year_encoder)

    know_data_, know_test_, vartype_info_ = salary_preprocessing(know_data_, know_test_, vartype_info_)

    for i in range(4):
        mean, std = know_data_[i]['NM'].mean(), know_data_[i]['NM'].std()
        know_data_[i]['NM'] = (know_data_[i]['NM']-mean)/std
        know_test_[i]['NM'] = (know_test_[i]['NM']-mean)/std
        know_data_[i]['C'] = know_data_[i]['C'].astype('category')
        know_test_[i]['C'] = know_test_[i]['C'].astype('category')

    nl_encoders, tr_X, ts_X = get_nl_features(feature_per_column = training_args.feature_per_column)

    using_nl = ['자격증', '전공', '사용도구', '일자리변화','훈련'] 
    # ['자격증', '전공', '훈련', '유사직업', '일자리변화', '사용도구', '이전직업', '전직가능', '신직업', '변화예시']
    train_bow = get_bow(tr_X, using_nl, nl_encoders)
    test_bow = get_bow(ts_X, using_nl, nl_encoders)

    for i, year in enumerate(years):
        know_data_[i]['bow'] = train_bow[year]
        know_test_[i]['bow'] = test_bow[year]

    return know_data_, know_test_

# Train

## Neural Network

### model

In [ ]:
class SurveyModel(nn.Module):
    """
        Multilayer Perceptron (MLP) Class
    """
    def __init__(self,cdim=784,nmdim=784,bowdim=784, hdim=512, ydim=10, gdim=None):
        super(SurveyModel,self).__init__()
        self.lin_c = nn.Linear(cdim,hdim)
        self.lin_nm = nn.Linear(nmdim,hdim)
        self.lin_bow = nn.Linear(bowdim,2*hdim)
        self.dropout = nn.Dropout(0.4)
        self.clf = nn.Linear(4*hdim, ydim)
        self.glf = nn.Linear(4*hdim, gdim) if gdim is not None else None
        self.init_param() # initialize parameters
        
    def init_param(self):
        for m in self.modules():
            if isinstance(m,nn.Linear):
                nn.init.kaiming_normal_(m.weight.data)
                m.bias.data.fill_(0)

    def forward(self, c_input, nm_input, bow_input, w2v_output=None,**kwargs):
        c_vec = self.lin_c(c_input)
        nm_vec = self.lin_nm(nm_input)
        bow_vec = self.lin_bow(bow_input)

        comb = [c_vec,nm_vec,bow_vec] if w2v_output is None else [c_vec,nm_vec,bow_vec, w2v_output]

        comb_vec = torch.concat(comb, axis=1)
        comb_vec = self.dropout(F.relu(comb_vec))
        net = self.clf(comb_vec)
        net2 = self.glf(comb_vec) if self.glf is not None else None

        return net, net2

### function

In [ ]:
class SurveyDataset(Dataset): # year 별로 4개 만들기
    def __init__(self, X, y=None, g = None, mode = 'simple'): 
        self.df = X
        self.key = list(X.keys()) # C, NM, bow, y
        self.L = X[self.key[0]].shape[0]
        self.y = y
        self.g = g
        self.mode = mode
        print(self.key)
        # 같은 y 값 가지는 idx dictionary를 만들고, 0.4 확률로 이를 mix up 해서 내놓는
        if mode == 'augment':
            self.get_hash()

    def get_hash(self):
        self.hash = defaultdict(list)
        for i, y in enumerate(self.y):
            self.hash[y].append(i)

    def __len__(self):
        return self.L

    def __getitem__(self, idx):
        batch = {}
        if self.y is not None :
            batch['y'] = torch.LongTensor([self.y[idx]])

        if self.g is not None :
            batch['g'] = torch.LongTensor([self.g[idx]])

        for col in self.key :
            temp = torch.FloatTensor(self.df[col].iloc[idx])
            batch[f"{col}_input".lower()] = temp
        
        if self.mode == 'augment' and random.random() < 0.4 :
            if random.random() > 0.25 :
                # 같은 knowcode에 속하는 midx를 가져와 feature값의 평균을 학습에 활용
                midx = random.choice(self.hash[self.y[idx]])
                for col in self.key :
                    temp = torch.FloatTensor(self.df[col].iloc[midx])
                    batch[f"{col}_input".lower()] = temp * 0.5 + batch[f"{col}_input".lower()] * 0.5
            else :
                # 자연어 관련련 feature를 masking
                batch["bow_input"] = torch.zeros_like(batch["bow_input"]).float()

        return batch
    
    def dim_info(self):
        batch = {}
        for col in self.key :
            if col != 'w2v':
                batch[f"{col}dim".lower()] = self.df[col].shape[1]
        if self.y is not None :
            batch['ydim'] = len(np.unique(self.y))
        if self.y is not None :
            batch['gdim'] = len(np.unique(self.g))
        return batch

In [ ]:
def get_binarized_df(train, test):
    binarized_df = pd.get_dummies(train)
    TEST = pd.get_dummies(test)
    co_columns = [c for c in TEST.columns if c in binarized_df.columns]
    binarized_df, TEST = binarized_df[co_columns], TEST[co_columns]
    return binarized_df, TEST

def get_data_loader(train, test, train_index = None, test_index = None, training_args = None):
    le = LabelEncoder()
    Y = le.fit_transform(train['y'])

    # knowcode의 앞자리를 대분류로 보아 같이 예측하게 함
    gle = LabelEncoder()
    G = None # gle.fit_transform(train['y'].apply(lambda x : x//1000))

    if train_index is not None :
        y_train, y_valid = Y[train_index], Y[test_index]
        g_train, g_valid = None, None # G[train_index], G[test_index]

        X_train = {key: value.iloc[train_index,:] for key, value in train.items() if key in ['C','NM','bow']} # 'w2v'
        X_valid = {key: value.iloc[test_index,:] for key, value in train.items() if key in ['C','NM','bow']} # 'w2v'

        train_set, valid_set = SurveyDataset(X_train, y_train, g_train, mode=training_args.mode), SurveyDataset(X_valid, y_valid, g_valid)
        train_iter = DataLoader(train_set,batch_size=training_args.batch_size,pin_memory=False, shuffle=True,num_workers=1)
        valid_iter = DataLoader(valid_set,batch_size=training_args.batch_size,shuffle=False,num_workers=1)

    else :

        X_train = {key: value.reset_index(drop=True) for key, value in train.items() if key in ['C','NM','bow']} # 'w2v'

        train_set = SurveyDataset(X_train, Y, G, mode=training_args.mode)
        train_iter = DataLoader(train_set,batch_size=training_args.batch_size,shuffle=True,num_workers=1)
        valid_iter = None

    X_test = {key: value for key, value in test.items() if key in ['C','NM','bow']} # ,'w2v'
    test_set = SurveyDataset(X_test)
    test_iter = DataLoader(test_set,batch_size=training_args.batch_size,shuffle=False,num_workers=1)

    return train_iter, valid_iter, test_iter, le, gle    

def get_y2g(le,gle):
    Y = le.inverse_transform(list(range(len(le.classes_))))
    G = [y//1000 for y in Y]
    g = gle.transform(G)
    g = list(Counter(g).values())
    return g 

In [ ]:
def total_step(year, train, test, train_index = None, test_index = None, training_args = None):
    if 'C' in train :
        train['C'], test['C'] = get_binarized_df(train['C'], test['C'])

    train_iter, valid_iter, test_iter, le, gle = get_data_loader(train, test, train_index = train_index, test_index = test_index, training_args = training_args)

    dim_info = train_iter.dataset.dim_info()
    dim_info['gdim'] = None
    model = SurveyModel(hdim=training_args.hdim,**dim_info).to(training_args.device)
    loss = nn.CrossEntropyLoss()

    optm = optim.Adam([{'params':model.parameters()}],lr=training_args.learning_rate)

    scheduler = optim.lr_scheduler.MultiStepLR(optimizer=optm, milestones=training_args.MILESTONE, gamma=0.8)

    for epoch in range(training_args.epochs):
        print('epoch',epoch)

        model, _ = train_one_epoch(train_iter, model, optm, loss, scheduler, le, None, training_args) # y2g if epoch >= WARMUP else None)
        if valid_iter is not None :
            _, result = valid_one_epoch(valid_iter, model, loss, None, mode='valid', training_args = training_args)# y2g if epoch >= WARMUP else None, mode='valid')

        # early stopping
    PRED, gPRED = valid_one_epoch(test_iter, model, None, None, mode='test', training_args = training_args) # y2g, mode='test')
    return le, PRED, gPRED, gle

In [ ]:
def get_mixed_pred(g_pred,y_pred, y2g, training_args):
    g_pred = F.softmax(g_pred,dim=1)
    g_ = torch.repeat_interleave(g_pred,torch.tensor(y2g).to(training_args.device),dim=1)
    y_pred = F.softmax(y_pred,dim=1)
    return torch.log(y_pred * g_+ 1e-8 * torch.ones_like(y_pred))

In [ ]:
def get_loss(pred, true, do = False):
    if len(true.size()) > 1 :
        # label smoothing 사용한 경우
        pred2 = pred.log_softmax(dim=-1) if do else pred
        return torch.mean(torch.sum(-true*pred2, dim=-1))
    else :
        # CrossEntropyLoss 사용한 경우
        criterion = nn.CrossEntropyLoss()
        return criterion(pred,true)

In [ ]:
def train_one_epoch(train_iter, model, optm, loss, scheduler, le, y2g, training_args):
    PRED, ANSWER = [], []
    gPRED, gANSWER = [], []
    total_loss = 0
    large_code, C = prepare_smoothing(le)
    model.train()

    for step, batch in enumerate(tqdm(train_iter)):

        s_input = {key:value.to(training_args.device) for key, value in batch.items()}

        y_true = get_smooth_target(batch['y'], large_code, C, factor=0.1).to(training_args.device)
        
        if 'g' in batch :
            g_true = batch['g'].flatten().to(training_args.device)

        y_pred, g_pred = model(**s_input)

        loss = get_loss(y_pred,y_true, do = True) / training_args.grad_accumulation
        
        if g_pred is not None :
            loss_g = get_loss(g_pred,g_true, do = True)
            loss = loss + loss_g * 1.0
            gPRED.extend(g_pred.argmax(axis=1).detach().cpu().numpy())
            gANSWER.extend(batch['g'].flatten().detach().cpu().numpy())

        if y2g is not None :
            y_pred = get_mixed_pred(g_pred,y_pred,y2g)
            loss += get_loss(y_pred,y_true) 

        total_loss += loss.item()

        loss.backward()

        if step % training_args.grad_accumulation == 0 :
            optm.step()
            model.zero_grad()
            

        PRED.extend(y_pred.argmax(axis=1).detach().cpu().numpy())
        ANSWER.extend(batch['y'].flatten().detach().cpu().numpy())


    acc = accuracy_score(ANSWER,PRED)
    f1 = f1_score(ANSWER,PRED,average='macro')
    gacc = accuracy_score(gANSWER,gPRED)
    gf1 = f1_score(gANSWER,gPRED,average='macro')

    print(f"train loss : {total_loss}, train accuracy : {acc}, train f1 score : {f1}")
    print(f"group accuracy : {gacc}, group f1 score : {gf1}")

    result = {'acc' : acc, 'f1' : f1, 'loss' : total_loss}
    scheduler.step()

    return model, result 

In [ ]:
def valid_one_epoch(valid_iter, model, loss, y2g, mode='valid', training_args =None):
    PRED, ANSWER = [], []
    gPRED, gANSWER = [], []

    total_loss = 0
    with torch.no_grad():
        model.eval()
        for batch in tqdm(valid_iter):
            s_input = {key:value.to(training_args.device) for key, value in batch.items()} # if key != 'y' or key != 'w2v_input'}
            
            y_pred, g_pred = model(**s_input)
            PRED.extend(y_pred.argmax(axis=1).cpu().numpy())

            if mode == 'valid' :
                
                y_true = batch['y'].flatten().to(training_args.device)

                if 'g' in batch :
                    g_true = batch['g'].flatten().to(training_args.device)
                    gANSWER.extend(g_true.cpu().numpy())
                    gPRED.extend(g_pred.argmax(axis=1).detach().cpu().numpy())
                    
                if y2g is not None :

                    y_pred = get_mixed_pred(g_pred,y_pred,y2g, training_args)
                    criterion = nn.NLLLoss()
                    loss = criterion(y_pred,y_true)

                else :
                    loss = get_loss(y_pred,y_true)
                    if g_pred is not None :
                        loss_g = get_loss(g_pred,g_true)
                        loss = loss + loss_g * 1.0


                total_loss += loss.item()
                ANSWER.extend(y_true.cpu().numpy())           

    if mode == 'valid':
        acc = accuracy_score(ANSWER,PRED)
        f1 = f1_score(ANSWER,PRED,average='macro')

        gacc = accuracy_score(gANSWER,gPRED)
        gf1 = f1_score(gANSWER,gPRED,average='macro')

        result = {'acc' : acc, 'f1' : f1, 'loss' : total_loss}
        print(f"valid loss : {total_loss}, valid accuracy : {acc}, valid f1 score : {f1}")
        print(f"group accuracy : {gacc}, group f1 score : {gf1}")

        return PRED, result

    else :
        return PRED, gPRED

In [ ]:
def prepare_smoothing(le):
    codes = le.classes_
    large_code = defaultdict(list)
    for c in codes :
        large_code[c//1000].append(c)
    return large_code, len(le.classes_)
# Label Smoothing!
def get_smooth_target(target, large_code, C, factor=0.2):
    B = target.size()[0]
    smths = []
    for idx in target :
        cls = large_code[idx//1000]
        smths.append([i for i,x in enumerate(target) if x in cls])
    return torch.zeros(B, C).scatter_(1,torch.tensor(smths),factor).scatter_(1,torch.tensor([[x] for x in (target)]),1-factor)

In [ ]:
def mixed_predict(le, pred, gPRED, gle) :
    a = [le.inverse_transform(p) for p in pred]
    b = [gle.inverse_transform(g) for g in gPRED]
    PRED = []
    for y_labels, g_labels in zip(a,b):
        for g in g_labels :
            out = False
            for y in y_labels :
                yg = y // 1000
                if yg == g :
                    PRED.append(y)
                    out = True
                    break
            if out :
                break
        if out :
            continue
        else :
            PRED.append(y_labels[0])
    if len(PRED) != len(a):
        print('wrong!!!!')
    return PRED

### full training

In [ ]:
def neuralnet_inference(training_args, file_name):
    temp = []
    augment = False

    know_data, know_test = get_data(training_args)
    
    for i, year in enumerate(tqdm(years)) :

        train = know_data[i]
        le, pred, gPRED, gle = total_step(year, train, know_test[i], train_index = None, test_index = None, training_args =training_args)
        temp.extend(le.inverse_transform(pred))
        torch.cuda.empty_cache()
        gc.collect()

    submission = pd.read_csv(os.path.join(data_dir,'sample_submission.csv'))

    submission['knowcode'] = temp
    submission['knowcode'] = submission['knowcode'].astype(int)
    submission.to_csv(os.path.join(training_args.output_dir, file_name), index=False)

## RandomForest

### function

In [ ]:
def get_soft_ensemble(PROBA, CLS):
    temp = []
    for year in (years):
        A = []
        for i in range(5):
            A.append(pd.DataFrame(PROBA[year][i],columns=CLS[year][i]))
        A = A[0] + A[1] + A[2] + A[3] + A[4]
        temp.extend(A.idxmax(axis=1).values)
    return temp

def get_RF_result(binarized_df, y, TEST, train_index = None, test_index = None):
    if train_index is not None :
        X_train, X_test = binarized_df.iloc[train_index,:], binarized_df.iloc[test_index,:]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    else :
        X_train, y_train = binarized_df, y
        X_test, y_test = None, None
    
    model = RandomForestClassifier(n_estimators=training_args.N_est, 
                                random_state=42,
                                max_depth = 256,
                                class_weight = "balanced_subsample",
                                n_jobs=-1)
    model.fit(X_train, y_train)

    if X_test is not None :
        y_pred = model.predict(X_test)
        print(f"f1 : {f1_score(y_test,y_pred,average='macro')}")

        pred, cls = model.predict_proba(TEST), model.classes_
        return pred, cls
    else :
        pred = model.predict(TEST)
        return pred

def RandomForest_inference(training_args, file_name):

    know_data, know_test = get_data(training_args)
    train_data, test_data = defaultdict(dict), defaultdict(dict)

    for i, year in enumerate(years):
        train_data[year]['X'] = pd.concat([know_data[i][c].reset_index(drop=True) for c in ['C','NM','bow']], axis=1)
        train_data[year]['y'] = know_data[i]['y']
        test_data[year]['X'] = pd.concat([know_test[i][c].reset_index(drop=True) for c in ['C','NM','bow']], axis=1)

    del know_data, know_test

    if training_args.cv > 0 :
        kf = StratifiedKFold(n_splits=training_args.cv)

    PROBA, CLS = {}, {}
    temp = []
    for year in tqdm(years):
        rf_predicts, rf_cls = [], []
        binarized_df, TEST = get_binarized_df(train_data[year]['X'], test_data[year]['X'])
        del train_data[year]['X'], test_data[year]['X']
        if training_args.cv == 0 :
            pred = get_RF_result(binarized_df, train_data[year]['y'], TEST, train_index = None, test_index = None)
            temp.extend(pred)

        if training_args.cv > 0 :
            for train_index, test_index in tqdm(kf.split(binarized_df,y=train_data[year]['y'])):

                pred, cls = get_RF_result(binarized_df, train_data[year]['y'], TEST, train_index = train_index, test_index = test_index)
                rf_predicts.append(pred)
                rf_cls.append(cls)
                gc.collect()

            PROBA[year] = rf_predicts
            CLS[year] = rf_cls
            
    if training_args.cv > 0 :
        temp = get_soft_ensemble(PROBA, CLS)

    submission = pd.read_csv(os.path.join(data_dir,'sample_submission.csv'))
    submission['knowcode'] = temp
    submission['knowcode'] = submission['knowcode'].astype(int)
    submission.to_csv(os.path.join(training_args.output_dir, file_name), index=False)

# 실행 코드

## Neural Network

In [ ]:
class TRAINING_ARGS :
    batch_size = 128
    hdim = 256
    epochs = 12
    learning_rate = 1e-3
    feature_per_column = 800
    grad_accumulation = 1
    device = 'cuda'
    MILESTONE = [6, 8, 10]
    WEIGHT_DECAY = 1e-2
    WARMUP = 6
    output_dir = '/content/drive/MyDrive/ensemble'
    mode = 'simple'

training_args = TRAINING_ARGS()

if not os.path.exists(training_args.output_dir):
    print(f'make new dir {training_args.output_dir}')
    os.mkdir(training_args.output_dir)

In [ ]:
# 1시간 33분 소요
neuralnet_inference(training_args, 'submission_hdim256.csv')
training_args.hdim = 512
neuralnet_inference(training_args, 'submission_hdim512.csv')
training_args.mode = 'augment'
neuralnet_inference(training_args, 'submission_augment.csv')

  0%|          | 0/4 [00:00<?, ?it/s]

['C', 'NM', 'bow']
['C', 'NM', 'bow']
epoch 0


  0%|          | 0/75 [00:00<?, ?it/s]

train loss : 346.86891174316406, train accuracy : 0.13999578325954037, train f1 score : 0.11630453976029118
group accuracy : nan, group f1 score : nan
epoch 1


  0%|          | 0/75 [00:00<?, ?it/s]

...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f40acabf5f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f40acabf5f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

train loss : 6.198395766317844, train accuracy : 0.9833764314739564, train f1 score : 0.9834017246471086
group accuracy : nan, group f1 score : nan
epoch 11


  0%|          | 0/64 [00:00<?, ?it/s]

train loss : 5.622910715639591, train accuracy : 0.9868242827238025, train f1 score : 0.9868770232117228
group accuracy : nan, group f1 score : nan


  0%|          | 0/64 [00:00<?, ?it/s]

## RandomForest

In [ ]:
class TRAINING_ARGS :

    output_dir = '/content/drive/MyDrive/ensemble'
    N_est = 200
    feature_per_column = 600
    cv = 5 

training_args = TRAINING_ARGS()

In [ ]:
RandomForest_inference(training_args, 'rf_5fold.csv')

erase na row 0
erase na row 34
erase na row 9
erase na row 1
already drop 3064 in 1
0
0
0
0
0 train data null value check 0
0 test data null value check 0
1 train data null value check 0
1 test data null value check 35
2 train data null value check 0
2 test data null value check 15
3 train data null value check 0
3 test data null value check 0
done!
0 bq4_1a min_df = 3 vocab size 553
0 bq5_2 min_df = 6 vocab size 572
0 bq19_1 min_df = 11 vocab size 757
0 bq30 min_df = 3 vocab size 387
0 bq31 min_df = 10 vocab size 574
0 bq32 min_df = 2 vocab size 595
0 bq33 min_df = 3 vocab size 458
0 bq34 min_df = 2 vocab size 199
0 bq38_1 min_df = 2 vocab size 508
1 bq4_1a min_df = 4 vocab size 511
1 bq5_2 min_df = 6 vocab size 541
1 bq28_1 min_df = 11 vocab size 652
1 bq29 min_df = 3 vocab size 519
1 bq30 min_df = 9 vocab size 532
1 bq31 min_df = 3 vocab size 476
1 bq32 min_df = 4 vocab size 535
1 bq33 min_df = 2 vocab size 495
1 bq37_1 min_df = 2 vocab size 576
2 bq4_1a min_df = 4 vocab size 503
2 

  0%|          | 0/4 [00:00<?, ?it/s]

0it [00:00, ?it/s]

f1 : 0.658802906681513
f1 : 0.678341536721156
f1 : 0.6875244969109782
f1 : 0.685408490060178
f1 : 0.6552847457041266


0it [00:00, ?it/s]

f1 : 0.6813945604065181
f1 : 0.68659514261881
f1 : 0.6785556250711697
f1 : 0.6710836084630596
f1 : 0.6663503594896191


0it [00:00, ?it/s]

f1 : 0.6010344742871625
f1 : 0.5677779741521385
f1 : 0.593877709117519
f1 : 0.5950338419110294
f1 : 0.5934019428529171


0it [00:00, ?it/s]

f1 : 0.6739593222833447
f1 : 0.6649195207854426
f1 : 0.6461208251711045
f1 : 0.6732468338613591
f1 : 0.6827580137636003


In [ ]:
training_args.N_est, training_args.cv = 100, 0
RandomForest_inference(training_args, 'rf_full.csv')

erase na row 0
erase na row 34
erase na row 9
erase na row 1
already drop 3064 in 1
0
0
0
0
0 train data null value check 0
0 test data null value check 0
1 train data null value check 0
1 test data null value check 35
2 train data null value check 0
2 test data null value check 15
3 train data null value check 0
3 test data null value check 0
done!
0 bq4_1a min_df = 3 vocab size 553
0 bq5_2 min_df = 6 vocab size 572
0 bq19_1 min_df = 11 vocab size 757
0 bq30 min_df = 3 vocab size 387
0 bq31 min_df = 10 vocab size 574
0 bq32 min_df = 2 vocab size 595
0 bq33 min_df = 3 vocab size 458
0 bq34 min_df = 2 vocab size 199
0 bq38_1 min_df = 2 vocab size 508
1 bq4_1a min_df = 4 vocab size 511
1 bq5_2 min_df = 6 vocab size 541
1 bq28_1 min_df = 11 vocab size 652
1 bq29 min_df = 3 vocab size 519
1 bq30 min_df = 9 vocab size 532
1 bq31 min_df = 3 vocab size 476
1 bq32 min_df = 4 vocab size 535
1 bq33 min_df = 2 vocab size 495
1 bq37_1 min_df = 2 vocab size 576
2 bq4_1a min_df = 4 vocab size 503
2 

  0%|          | 0/4 [00:00<?, ?it/s]

# Ensemble

## hard voting

In [ ]:
cand = glob(f"{training_args.output_dir}/*.csv")
print(cand)

['/content/drive/MyDrive/ensemble/submission_hdim256.csv', '/content/drive/MyDrive/ensemble/submission_hdim512.csv', '/content/drive/MyDrive/ensemble/submission_augment.csv', '/content/drive/MyDrive/ensemble/rf_5fold.csv', '/content/drive/MyDrive/ensemble/rf_full.csv']


In [ ]:
temp = pd.concat([pd.read_csv(pth, index_col='idx') for pth in cand], axis=1).mode(axis=1)[0].astype(int).values
temp

array([ 29401,  28402, 416304, ..., 833001, 140204, 833001])

# Submission

In [ ]:
submission = pd.read_csv(os.path.join(data_dir,'sample_submission.csv'))

submission['knowcode'] = temp
submission['knowcode'] = submission['knowcode'].astype(int)
submission.to_csv('/content/final_result.csv', index=False)